In [3]:
import csv
import time
import pandas as pd
from selenium import webdriver
from datetime import date
from bs4 import BeautifulSoup as soup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
import re 
import numpy as np
def use_selenium_automation():
    driver =webdriver.Firefox()
    return driver
def crawl_and_scrap_webpages_yt(driver,url):
    driver.get(url) 
    extracted_page_source = driver.page_source
    time.sleep(1)
    pincode = driver.find_element_by_id('tbxSearch').send_keys("560076") 
    time.sleep(1)
    #pincode.send_keys("560003") 
    findstore = driver.find_element_by_id("btnFindStore")
    findstore.click()
    SCROLL_PAUSE_TIME=1
    try:
        #time.sleep(1)
        store = driver.find_element_by_xpath("//div[starts-with(@id, 'loc_')]")
        time.sleep(1)
        store.click()
        time.sleep(1)
        category = driver.find_element_by_id("txtSearch")
        category.send_keys(Keys.CONTROL + "a");
        category.send_keys(Keys.DELETE)
        category.send_keys("fruits and vegetables")
        category.send_keys(Keys.ENTER)
        time.sleep(2)
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        time.sleep(1)
        while True:
            driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
            time.sleep(1)
            try:
                #wait = WebDriverWait(driver, 10)
                #downloads = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[4]/center/div/div/center/div/div[3]/div[2]/div[4]/div/div/div/div[3]/div[2]/div[2]")))
                #ActionChains(driver).move_to_element(downloads).click().perform()
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[4]/center/div/div/center/div/div[3]/div[2]/div[4]/div/div/div/div[3]/div[2]/div[2]"))).click()
                print("LOAD MORE RESULTS button clicked")
            except TimeoutException:
                print("No more LOAD MORE RESULTS button to be clicked")
                break
            
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            time.sleep(1)
            if new_height == last_height:
                break
            last_height = new_height
        
        res = driver.execute_script( "return document.documentElement.outerHTML" )
        page_soup = soup( res, "html.parser" )
        containers = page_soup.findAll("div", {"class": "mtb-desc"})
        titles = [x.text.replace(',','') for x in containers]  
        sp=page_soup.findAll("span",{"class" : "mtb-price"})
        price=[]
        for e in range(0,len(sp),2):
            try:
                price.append(sp[e].findAll("label",{"class" : "mtb-ofr SaveOnMRP"})[0].findAll("span",{"class" : "sp_amt"})[0].text.replace(',',''))    
            except:
                price.append("")
        quantity=[]
        sp=page_soup.findAll("div",{"class" : "hyc-showcase-bottom"})
        for i in range(len(sp)):
            try:
                quantity.append(sp[i].findAll("option",{"class" : "VariantPropertyValue"})[0].text.replace(',',''))
            except:
                quantity.append("")
        return titles,price,quantity
    except:
        print("error")
        return [],[],[]
if __name__=="__main__":
    today = date.today()
    d1 = today.strftime("%Y-%m-%d")
    date_1 = today.strftime("%d_%m_%Y")
    driver=use_selenium_automation()  
    urls=["https://www.simplinamdharis.com/"]
    for url in urls:
        title, price, quantity = crawl_and_scrap_webpages_yt(driver,url)
    driver.close()
    len(title)
    len(price)
    len(quantity)
    name_dict = {'Name': title,
            'Price(Rs)': price,
            'Quantity': quantity
          }
    df = pd.DataFrame(name_dict)
    df['Website']='simplinamdharis'
    df['date']=d1
    df['location']='hyderabad'
    df['New_quantity']=df[df['Quantity'] == ''].Name.str.split(r"([0-9])",n=1,expand=True)[1]+df[df['Quantity'] == ''].Name.str.split(r"([0-9])",n=1,expand=True)[2]
    df['Quantity']=df['Quantity'].replace(r'', np.nan, regex=True)
    df.Quantity.fillna(df.New_quantity, inplace=True)
    df.Quantity.fillna("1 units", inplace=True)
    df=df.drop(['New_quantity'], axis=1)
    df['Name'].update(df.Name.str.split(r"([0-9])",n=1,expand=True)[0])
    df.loc[df['Name'].str.contains('kg', case=False), 'Quantity'] = '1 kg'
    #df.to_csv('namdhari_bangalore5_unprocessed.csv', index=False)
    df.to_csv(date_1+'.csv',mode='a', header=False,index=False)

LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked


In [4]:
df

,Name,Price(Rs),Quantity,Website,date,location
0,Marjoram,14,25 G Pb,simplinamdharis,2020-06-26,hyderabad
1,Thyme,15,25 G Pb,simplinamdharis,2020-06-26,hyderabad
2,Oregano,17,25 G Pb,simplinamdharis,2020-06-26,hyderabad
3,Sage,15,25 G Pb,simplinamdharis,2020-06-26,hyderabad
4,Rosemary,15,25 G Pb,simplinamdharis,2020-06-26,hyderabad
5,Basil Lemon,10,25 G Pb,simplinamdharis,2020-06-26,hyderabad
6,Wheat Grass,9,25 Gpb,simplinamdharis,2020-06-26,hyderabad
7,Basil Thai,10,25 G Pb,simplinamdharis,2020-06-26,hyderabad
8,Mint,9,100G Bunch,simplinamdharis,2020-06-26,hyderabad
9,Basil Italian,10,25 G Pb,simplinamdharis,2020-06-26,hyderabad
